# Multi-agent for Resume Reviewing - Modified Resume based on Job Application


In [1]:
#!pip install crewai crewai_tools langchain_community

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [6]:
from crewai import Agent, Task, Crew , LLM

ImportError: cannot import name 'CompletionTokensDetails' from 'openai.types.completion_usage' (C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\openai\types\completion_usage.py)

In [4]:
import os
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
gemini_api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key

TypeError: str expected, not NoneType

In [5]:
llm=LLM(
        model="gemini/gemini-1.5-flash",
        temperature=0.5,
        # verbose=True,    
        )

NameError: name 'LLM' is not defined

## crewAI Tools

In [18]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool , PDFSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
pdf_tool = PDFSearchTool(config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini-1.5-flash",
                temperature=0.5,

                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                title="Embeddings",
                
            ),
        ),
    ))

## Creating Agents

In [19]:
resume_analyst_agent = Agent(
    role="Senior Resume Analytics Agent",
    goal="Read ({resume}) file provided by customers",
    backstory="Specializing in analyzing resume, this agent "
              "uses the ({resume}) provided by customers"
              "to provide crucial insights on resume"
              "the resume analyst agent will play a critical role to help other agent optimize resume",

    verbose=True,
    allow_delegation=True,
    tools = [pdf_tool]
)

In [20]:
job_analyst_agent = Agent(
    role="Senior Job Application Analytics Agent",
    goal="Read ({joburl}) provided by customers",
    backstory="Specializing in analyzing job application and keywords, this agent "
              "uses the ({joburl}) provided by customers"
              "to provide crucial insights on the job, the requirements and the keywords"
              "the job analyst agent will play a critical role to help other agent optimize resume following ATS standard",

    verbose=True,
    allow_delegation=True,
    tools = [search_tool,scrape_tool]
)

In [21]:
optimized_resume_analyst_agent = Agent(
    role="Senior Resume Optimization Agent",
    goal="Read ({joburl}) and ({resume}) provided by customers",
    backstory="Specializing in optimization resume, this agent"
              "uses the ({joburl}) provided by customers"
              "use ({resume}) file provided by customers"
              "use insight provided by other agents to optimize resume"
              "to provide new optimized points on resume"
              "the optimized resume analyst agent will play an executive role, taking other agents insight into account and provide answer for users",

    verbose=True,
    allow_delegation=True,
    tools = [pdf_tool,search_tool,scrape_tool]
)

## Creating Tasks

In [22]:
# Task for resume_analyst_agent
resume_analytics_task = Task(
    description=(
        "Read ({resume}) file provided by customers "
        "Understanding their background, experiences"
    ),
    expected_output=(
        "Insights and keywords about customer's experiences"
        "opportunities to improve."
    ),
    agent=resume_analyst_agent,
)


In [23]:
# Task for job_analyst_agent
job_analyst_task = Task(
    description=(
        "extract crucial keywords, requirements based on "
        "the ({joburl}) provided by customers "
    ),
    expected_output=(
"the list of requirements and keywords from the ({joburl}) that is must-have to pass ATS system"
    ),
    agent=job_analyst_agent,
)


In [24]:
# Task for optimized_resume_analyst_agent
optimized_resume_task = Task(
    description=(
        "Evaluate customer's resume by "
         "using the ({joburl}) provided by customers"
         "using the ({resume}) provided by customers"
          "use insight provided by other agents to optimize resume"
        "Provide a detailed analysis on how to tailor customer's resume to pass the ATS "
        "and suggest to rewrite some of their experiences if needed."
    ),
    expected_output=(
        "A comprehensive rewrite all the experiences on customer's resume following ATS standard and using keywords extracted from the job description provided "
    ),
    agent=optimized_resume_analyst_agent,
)

manager_llm: CrewAI will create a manager in the background for us. We just need to indicate the model here

In [25]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
job_analytics_crew = Crew(
    agents=[resume_analyst_agent,
            job_analyst_agent,
            optimized_resume_analyst_agent],

    tasks=[resume_analytics_task,
           job_analyst_task,
           optimized_resume_task],

    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

2025-02-12 21:24:38,203 - 22292 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [30]:
# Another Example
resume_optimization_input = {
    'joburl': 'https://www.google.com/about/careers/applications/jobs/results/110690555461018310-software-engineer-iii-infrastructure-core?src=Online/LinkedIn/linkedin_us&utm_source=linkedin&utm_medium=jobposting&utm_campaign=contract&hl=en_US',
    'resume': 'MY_RESUME.pdf'
}

In [31]:
### this execution will take some time to run
result = job_analytics_crew.kickoff(inputs=resume_optimization_input)

# Agent: Crew Manager
## Task: Read (MY_RESUME.pdf) file provided by customers Understanding their background, experiences
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 2 validation errors for PDFSearchToolSchema
query
  Input should be a valid string [type=string_type, input_value={'description': 'Extract ...llsets.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
pdf
  Input should be a valid string [type=string_type, input_value={'description': 'MY_RESUME.pdf', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type.
 Tool Search a PDF's content accepts these inputs: Tool Name: Search a PDF's content
Tool Arguments: {'query': {'description': "Mandatory query you want to use to search the PDF's content", 'type': 'str'}, 'pdf': {'description': 'Mandatory pdf path you want to search', 'type': 'str'}}
Tool D

In [16]:
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))

TypeError: Markdown expects text, not CrewOutput(raw='I am unable to complete the task due to errors accessing the necessary job description.  Please provide a valid job description text.\n```', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Read (MY_RESUME.pdf) file provided by customers Understanding their background, experiences', name=None, expected_output="Insights and keywords about customer's experiencesopportunities to improve.", summary='Read (MY_RESUME.pdf) file provided by customers Understanding their background, experiences...', raw="I require the MY_RESUME.pdf file to analyze the customer's background and experience and provide insights and keywords. Please provide the resume so I can complete the task.  My analysis will include a summary of key experiences, skills, potential areas for improvement, and relevant keywords to highlight opportunities.\n```", pydantic=None, json_dict=None, agent='Crew Manager', output_format=<OutputFormat.RAW: 'raw'>), Ta